In [1]:
import pandas as pd
import numpy as np
#import datatable as dt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import optuna

print (xgb.__version__)

1.5.0


## Memory Reduction

In [2]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [3]:
train = pd.read_csv('../input/train.csv', nrows=10000)
train = reduce_memory_usage(train)
test = pd.read_csv('../input/test.csv', nrows=5000)
test = reduce_memory_usage(test)

ss = pd.read_csv('../input/sample_submission.csv')
ss = reduce_memory_usage(ss)

Mem. usage decreased to 5.04 Mb (77.0% reduction)
Mem. usage decreased to 2.52 Mb (76.9% reduction)
Mem. usage decreased to 2.86 Mb (62.5% reduction)


In [4]:
## Based on the way this is loaded I'm expecting these to be blank

bool_cols_train = []
for i, col in enumerate(train.columns):
    if train[col].dtypes == bool:
        bool_cols_train.append(i)

In [5]:
## Just checking that it was empty
print (bool_cols_train)

[]


In [6]:
print("Train set shape", train.shape, "\n", "Test set shape", test.shape)

Train set shape (10000, 287) 
 Test set shape (5000, 286)


In [7]:
train.head()
### I really just want to looks at f276 through to f284

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
0,0,0.205933,0.410889,0.176758,0.223633,0.423584,0.476074,0.413574,0.611816,0.534668,...,0,1,0,0,0,0,0,0,0,1
1,1,0.181030,0.473145,0.011734,0.213623,0.619629,0.441650,0.230347,0.686035,0.281982,...,0,1,0,0,0,0,0,0,0,1
2,2,0.182617,0.307373,0.325928,0.207153,0.605469,0.309814,0.493408,0.750977,0.536133,...,0,0,0,1,1,0,0,0,0,1
3,3,0.180298,0.494629,0.008369,0.223633,0.760742,0.439209,0.432129,0.776367,0.483887,...,0,0,0,0,1,0,0,0,0,1
4,4,0.177124,0.495605,0.014259,0.548828,0.625488,0.562500,0.117188,0.561035,0.077087,...,0,1,1,0,1,0,0,1,0,1


## Modeling

In [8]:
## Will need some slight changes here based on pd.read_csv and still having "id"
X = train.drop(columns=["id", "target"]).copy()
y = train["target"].copy()

X_test = test.drop(columns=["id"]).copy()


# X = train.drop('target', axis=1).copy()
# y = train['target'].copy()
# X_test = test.copy()

del train
del test

In [9]:
### I talk about this in the introduction section of version 4 of https://www.kaggle.com/joecooper/tps-oct-joes-sandpit
### It is relevant here as well. 
## To be more specific I think it is also relevant to the perceived benefits of Target Encoding potentially showing improvments

# For now I leave this section of code in so we can do a like for like comparison of v1 of this notebook

X['std'] = X.std(axis=1)
X['min'] = X.min(axis=1)
X['max'] = X.max(axis=1)

X_test['std'] = X_test.std(axis=1)
X_test['min'] = X_test.min(axis=1)
X_test['max'] = X_test.max(axis=1)

In [10]:
## My new section of code for this notebook is to add StandardScaler
## Again I point to my commentary in  https://www.kaggle.com/joecooper/tps-oct-joes-sandpit where I run the scaler. I'm thinkning that the logic is similar to min/max/std FE

## Is running a scaler as useful as TargetEncoding?

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = pd.DataFrame (data=scaler.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame (data=scaler.transform(X_test), columns=X_test.columns)


### And now I'm changing my mind again. I think there are a few benefits to running Scalers !

### Some thought will have to go into the inclusion of the std/min/max features being but through a scaler 

In [11]:
X.head(10)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f278,f279,f280,f281,f282,f283,f284,std,min,max
0,-0.094360,-0.428711,0.489746,-0.824707,-1.380859,0.887207,0.075439,-0.580566,0.570801,-0.902344,...,-0.428467,-0.375000,-0.403076,-0.482422,-0.412354,-0.456543,-0.408447,0.227295,0.0,0.0
1,-0.596680,0.193115,-0.974121,-0.988770,0.268799,0.293701,-1.309570,0.514160,-1.357422,-0.124573,...,-0.428467,-0.375000,-0.403076,-0.482422,-0.412354,-0.456543,-0.408447,-0.058411,0.0,0.0
2,-0.564453,-1.462891,1.812500,-1.094727,0.149658,-1.978516,0.679199,1.471680,0.582031,0.287598,...,-0.428467,2.667969,2.480469,-0.482422,-0.412354,-0.456543,-0.408447,-1.820312,0.0,0.0
3,-0.611328,0.407959,-1.003906,-0.824707,1.456055,0.251709,0.215698,1.845703,0.183594,0.067017,...,-0.428467,-0.375000,2.480469,-0.482422,-0.412354,-0.456543,-0.408447,0.441650,0.0,0.0
4,-0.674805,0.417725,-0.951660,4.511719,0.318115,2.376953,-2.166016,-1.330078,-2.921875,-0.808594,...,2.335938,-0.375000,2.480469,-0.482422,-0.412354,2.191406,-0.408447,0.274902,0.0,0.0
5,0.897461,-1.077148,-0.887207,-0.872559,-0.527832,-0.017548,0.979980,-0.335938,-0.908203,-0.906250,...,2.335938,-0.375000,2.480469,-0.482422,-0.412354,-0.456543,-0.408447,-0.082214,0.0,0.0
6,-0.141113,-0.189819,0.523438,-0.585938,0.499023,3.597656,1.334961,1.586914,0.850586,2.785156,...,-0.428467,-0.375000,-0.403076,-0.482422,-0.412354,-0.456543,-0.408447,-0.653809,0.0,0.0
7,-0.517578,0.356689,-0.240723,0.501953,1.131836,-2.044922,-0.133179,0.009766,-1.031250,1.669922,...,-0.428467,-0.375000,-0.403076,2.074219,-0.412354,2.191406,-0.408447,0.441650,0.0,0.0
8,-0.918457,2.027344,-0.920898,-1.044922,-1.004883,0.470459,0.234253,-0.292725,1.129883,-0.875000,...,-0.428467,-0.375000,-0.403076,-0.482422,-0.412354,-0.456543,-0.408447,0.584473,0.0,0.0
9,-0.599121,-0.028748,0.432373,1.351562,2.832031,1.156250,0.464844,0.024185,-1.258789,-0.349609,...,-0.428467,-0.375000,-0.403076,-0.482422,-0.412354,-0.456543,-0.408447,-1.724609,0.0,0.0


In [12]:
params = {
    'max_depth': 6,
    'n_estimators': 9500,
    'subsample': 0.7,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.6000000000000001,
    'min_child_weight': 56.41980735551558,
    'reg_lambda': 75.56651890088857,
    'reg_alpha': 0.11766857055687065,
    'gamma': 0.6407823221122686,
    'booster': 'gbtree',
    'eval_metric': 'auc',
#    'tree_method': 'gpu_hist',
#    'predictor': 'gpu_predictor',
    'use_label_encoder': False
    }

In [14]:
%%time
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=786)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
#    params['learning_rate']=0.007
    params['learning_rate']=0.01
    model1 = XGBClassifier(**params)
    
    model1.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False)
    
#    params['learning_rate']=0.05
    params['learning_rate']=0.05
    model2 = XGBClassifier(**params)
    
    model2.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False,
              xgb_model=model1)
    
#    params['learning_rate']=0.01
    params['learning_rate']=0.007
    model3 = XGBClassifier(**params)
    
    model3.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False,
              xgb_model=model2)
    
    pred_valid = model3.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('||'*40)
    
    test_preds = model3.predict_proba(X_test)[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

Fold: 1 Score: 0.8421342910537025
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 2 Score: 0.8390863619938761
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 3 Score: 0.8318814380619776
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 4 Score: 0.847216174743929
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Fold: 5 Score: 0.8337051906882513
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Overall Validation Score: 0.8388046913083473
CPU times: user 9min 50s, sys: 2min 41s, total: 12min 32s
Wall time: 1min 49s
